<a href="https://colab.research.google.com/github/Dulon18/Machine_Learning_project/blob/main/Credit_card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Work Flow**


Creadit Card Data ----> Data preprocessing -->Data Analysis ---> Train Test Data --> Logistic Regration Model ---> Evaluation

In [114]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [115]:
# Loading the dataset
creditcard_dataset = pd.read_csv('/content/creditcard.csv')
creditcard_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [116]:
creditcard_dataset.shape

(284807, 31)

In [117]:
#dataset information
creditcard_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [118]:
#check missing value
creditcard_dataset.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [119]:
# Distribution of legit transaction & fraudulent transaction
creditcard_dataset['Class'].value_counts()



0    284315
1       492
Name: Class, dtype: int64

# **This Dataset is highly unbalanced**


0--> Normal transaction

1--> Fraudulent transaction

In [120]:
#Seperating the data for analysis
legit = creditcard_dataset[creditcard_dataset.Class == 0]
fraud = creditcard_dataset[creditcard_dataset.Class == 1]


In [121]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [122]:
#statistical measures of the data
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [123]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [124]:
# Compare the values for both transaction

creditcard_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


# **Under-Sampling**

Buliding a sample dataset containg similar distribution of normal transactions and fraud transactions.

Number of fraudulent Transactions -->492

In [125]:
legit_sample = legit.sample(n=492)

In [126]:
#Concatanating two dataframes

new_dataset =pd.concat([legit_sample,fraud],axis=0)
# axis 0 mean row & 1 means colomn

In [127]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
164219,116548.0,-2.671064,-1.755989,1.342040,1.360303,1.507190,-0.453708,-0.112573,0.172640,-0.258151,0.188011,0.686711,1.001924,1.333940,-0.171315,0.492736,0.265462,-0.874177,1.090050,0.788875,0.003263,0.175053,1.249761,1.539040,-0.399015,0.501706,-0.286118,0.572341,0.070731,88.50,0
21681,31805.0,1.312581,-0.686000,-0.320082,-0.841366,-0.380897,-0.248501,-0.203107,-0.236858,-0.948141,0.604739,-1.569336,-0.129799,1.386750,-0.094655,0.883724,-0.867700,-0.577319,1.066904,-0.172978,-0.185902,-0.615446,-1.488654,-0.075584,-0.983679,0.240404,0.890117,-0.074834,0.014613,102.42,0
165076,117182.0,1.915695,-0.200272,-0.664386,1.010571,0.099264,0.380483,-0.354467,0.092678,0.552912,0.296222,-0.036190,1.148867,0.920288,0.017721,-0.285564,0.818758,-1.330568,0.543272,0.259890,-0.089927,-0.204394,-0.546561,0.201021,-1.156141,-0.259192,-0.939764,0.046113,-0.038060,43.40,0
24508,33264.0,-1.098412,0.777459,1.136776,1.269698,0.000849,0.288053,0.327463,0.476333,-0.304851,0.193053,0.819247,0.240914,-1.433273,0.521890,-0.322415,-0.912093,0.494493,-0.015546,1.050694,-0.048679,-0.020055,0.326541,0.169719,-0.009999,-0.101159,-0.261602,0.053300,-0.139596,38.89,0
120022,75685.0,1.254968,0.035608,-0.418685,-0.361613,0.185110,-0.600309,0.364221,-0.202572,-0.344698,-0.047192,0.906117,0.968961,0.595760,0.522487,0.135048,0.406262,-0.728300,-0.293852,0.931545,0.078165,-0.355992,-1.155358,-0.006794,-0.454884,0.287889,0.764367,-0.118604,-0.011726,46.75,0


In [128]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,93434.292683,0.027127,-0.059806,-0.049997,-0.089665,-0.050304,0.013253,0.051441,0.018084,-0.071515,0.073250,0.013879,0.005794,-0.002833,-0.005326,-0.045278,0.001996,0.010029,0.041479,0.015991,-0.008247,-0.007198,0.020002,-0.004784,-0.014069,0.056876,-0.037861,0.012023,-0.013161,95.453089
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


# **Splitting data into Features and Targets**

In [129]:
x=new_dataset.drop(columns='Class',axis=1)
y =new_dataset['Class']

print(x,y)

            Time        V1        V2  ...       V27       V28  Amount
164219  116548.0 -2.671064 -1.755989  ...  0.572341  0.070731   88.50
21681    31805.0  1.312581 -0.686000  ... -0.074834  0.014613  102.42
165076  117182.0  1.915695 -0.200272  ...  0.046113 -0.038060   43.40
24508    33264.0 -1.098412  0.777459  ...  0.053300 -0.139596   38.89
120022   75685.0  1.254968  0.035608  ... -0.118604 -0.011726   46.75
...          ...       ...       ...  ...       ...       ...     ...
279863  169142.0 -1.927883  1.125653  ...  0.292680  0.147968  390.00
280143  169347.0  1.378559  1.289381  ...  0.389152  0.186637    0.76
280149  169351.0 -0.676143  1.126366  ...  0.385107  0.194361   77.89
281144  169966.0 -3.113832  0.585864  ...  0.884876 -0.253700  245.00
281674  170348.0  1.991976  0.158476  ...  0.002988 -0.015309   42.53

[984 rows x 30 columns] 164219    0
21681     0
165076    0
24508     0
120022    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Nam

In [130]:
#split the data into traning & testing data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.2, stratify =y,random_state =2)

In [131]:
print(x.shape,x_train.shape,x_test.shape)

(984, 30) (787, 30) (197, 30)


# **Model Traning**

In [132]:
model = LogisticRegression()

In [133]:
#traing the model with traing data
model.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# **Model Evoluation**

In [134]:
#Accuracy Score for training data
x_train_prediction =model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction,y_train)

In [135]:
print("Accuracy on Training Data : ", training_data_accuracy)

Accuracy on Training Data :  0.9453621346886912


In [136]:
#Accuracy Score for training data
x_test_prediction =model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction,y_test)

print("Accuracy on Training Data : ", test_data_accuracy)

Accuracy on Training Data :  0.9441624365482234
